<hr>

## Data Loads for Numeric Regression ##
- **demo.housing_prices_full**

In [2]:
# /* -------------------------------------------------------- */
# -- Perform this as SYSDBA.
# -- Load demo.housing_prices_full
# /* -------------------------------------------------------- */

import teradatasql, json

# load vars json
with open('../../vars.json', 'r') as f:
    session_vars = json.load(f)

name = session_vars['hierarchy']['SYSDBA']['username']
pwd = session_vars['hierarchy']['SYSDBA']['password']

bucket = session_vars['environment']['bucket']
access_key = session_vars['environment']['access_key']
secret = session_vars['environment']['secret']

with teradatasql.connect(host = session_vars['environment']['host'], 
                     user = name, 
                     password = pwd) as con:
    cur = con.cursor()
    
    try:
        cur.execute('DROP TABLE demo_ofs.housing_prices_full')
    except Exception as e:
        # Table already exists
        if str(e.args).find("3807") >= 1:
            pass
        else:
            raise

    qry = '''
    CREATE TABLE demo_ofs.housing_prices_full,

    STORAGE=TD_OFSSTORAGE

         (
          id BIGINT,
          mssubclass BIGINT,
          mszoning VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          lotfrontage FLOAT,
          lotarea BIGINT,
          street VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          alley VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          lotshape VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          landcontour VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          utilities VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          lotconfig VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          landslope VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          neighborhood VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          condition1 VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          condition2 VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          bldgtype VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          housestyle VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          overallqual BIGINT,
          overallcond BIGINT,
          yearbuilt BIGINT,
          yearremodadd BIGINT,
          roofstyle VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          roofmatl VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          exterior1st VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          exterior2nd VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          masvnrtype VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          masvnrarea FLOAT,
          exterqual VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          extercond VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          foundation VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          bsmtqual VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          bsmtcond VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          bsmtexposure VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          bsmtfintype1 VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          bsmtfinsf1 BIGINT,
          bsmtfintype2 VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          bsmtfinsf2 BIGINT,
          bsmtunfsf BIGINT,
          totalbsmtsf BIGINT,
          heating VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          heatingqc VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          centralair VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          electrical VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          _1stflrsf BIGINT,
          _2ndflrsf BIGINT,
          lowqualfinsf BIGINT,
          grlivarea BIGINT,
          bsmtfullbath BIGINT,
          bsmthalfbath BIGINT,
          fullbath BIGINT,
          halfbath BIGINT,
          bedroomabvgr BIGINT,
          kitchenabvgr BIGINT,
          kitchenqual VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          totrmsabvgrd BIGINT,
          functional VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          fireplaces BIGINT,
          fireplacequ VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          garagetype VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          garageyrblt BIGINT,
          garagefinish VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          garagecars BIGINT,
          garagearea BIGINT,
          garagequal VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          garagecond VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          paveddrive VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          wooddecksf BIGINT,
          openporchsf BIGINT,
          enclosedporch BIGINT,
          _3ssnporch BIGINT,
          screenporch BIGINT,
          poolarea BIGINT,
          poolqc VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          fence VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          miscfeature VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          miscval BIGINT,
          mosold BIGINT,
          yrsold BIGINT,
          saletype VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          salecondition VARCHAR(1024) CHARACTER SET UNICODE NOT CASESPECIFIC,
          saleprice BIGINT,
          part BIGINT)
    ;
    '''

    cur.execute(qry)
    
    qry = f'''
    Insert into demo_ofs.housing_prices_full 
    SELECT * FROM Antiselect (
              ON (SELECT *
                  FROM (
                    LOCATION = '/s3/{bucket}.s3.amazonaws.com/DEMO_OFS/HOUSING_PRICES_FULL/manifest.json'
                    AUTHORIZATION = '{{"Access_ID":"{access_key}","Access_Key":"{secret}"}}'
                    MANIFEST = 'TRUE'
                ) D)
              USING 
              Exclude ('location')
            ) AS dt;'''
    
    
    cur.execute(qry)